# JSON Mini-Project by Farkhod Narbayev

1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
# import packages:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np
import operator

In [2]:
# assign file url:
url = 'data_wrangling_json/data/world_bank_projects.json'

In [3]:
# read file and turn into dataframe:
with open(url) as json_file:
    json_data = json.load(json_file)

# creating dataframe by normalizing json_data:
df = json_normalize(json_data)

In [4]:
# get details of dataframe:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 55 columns):
_id.$oid                    500 non-null object
approvalfy                  500 non-null object
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 no

In [5]:
# create dataframe with relevant columns
df1 = df[['_id.$oid', 
         'boardapprovaldate', 
         'closingdate', 
         'borrower', 
         'countrycode', 
         'countryname', 
         'status',
         'theme1.Name', 
         'theme1.Percent', 
         'totalamt', 
         'totalcommamt', 
          'mjtheme', 
          'mjthemecode', 
          'mjtheme_namecode']]

In [6]:
# rename column names
df1.columns = ['ID', 
               'BoardApprovalDate', 
               'ClosingDate', 
               'Borrower', 
               'CountryCode', 
               'CountryName', 
               'Status', 
               'ThemeName', 
               'ThemePercent', 
               'TotalAmount', 
               'TotalCommAmount',
               'Theme', 
               'ThemeCode', 
               'Theme_Name_Code']

In [7]:
# check ids on uniqueness
df1['ID'].nunique()

500

In [8]:
# finding top 10 countries with the most projects, answer to the question 1
df1.groupby('CountryName').ID.count().sort_values(ascending=False).head(10)

CountryName
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: ID, dtype: int64

In [9]:
# check the first row of Theme_Name_Code column
df1.iloc[0].Theme_Name_Code

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]

In [10]:
# count number of themes in Theme_Name_Code column
count_theme = {}
for item in df1.Theme_Name_Code:
    for value in item:
        count_theme[value['code']] = count_theme.get(value['code'], 0) + 1

In [11]:
print(count_theme)

{'8': 210, '11': 250, '1': 38, '6': 168, '5': 77, '2': 199, '7': 130, '4': 146, '10': 216, '9': 50, '3': 15}


In [12]:
# sort the theme count in descending order
sorted_count_theme = sorted(count_theme.items(), key=operator.itemgetter(1), reverse=True)

In [13]:
print(sorted_count_theme[:10])

[('11', 250), ('10', 216), ('8', 210), ('2', 199), ('6', 168), ('4', 146), ('7', 130), ('5', 77), ('9', 50), ('1', 38)]


In [14]:
# create a dictionary of codes and names of themes with filled name values
code_name = {}
for item in df1.Theme_Name_Code:
    for value in item:
        if value['name'] is not '':
            code_name[value['code']] = value['name']

In [15]:
print(code_name)

{'8': 'Human development', '1': 'Economic management', '6': 'Social protection and risk management', '5': 'Trade and integration', '2': 'Public sector governance', '11': 'Environment and natural resources management', '7': 'Social dev/gender/inclusion', '4': 'Financial and private sector development', '10': 'Rural development', '9': 'Urban development', '3': 'Rule of law'}


In [16]:
# create new list of dictionaries and replace Theme_Name_Code with filled theme names
new_code = []
for item in df1.Theme_Name_Code:
    code = []
    for value in item:
        code.append({'code': value['code'], 'name': code_name[value['code']]})
    new_code.append(code)

In [17]:
df1.Theme_Name_Code = new_code

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [18]:
df1.Theme_Name_Code

0      [{'code': '8', 'name': 'Human development'}, {...
1      [{'code': '1', 'name': 'Economic management'},...
2      [{'code': '5', 'name': 'Trade and integration'...
3      [{'code': '7', 'name': 'Social dev/gender/incl...
4      [{'code': '5', 'name': 'Trade and integration'...
5      [{'code': '6', 'name': 'Social protection and ...
6      [{'code': '2', 'name': 'Public sector governan...
7      [{'code': '11', 'name': 'Environment and natur...
8      [{'code': '10', 'name': 'Rural development'}, ...
9      [{'code': '2', 'name': 'Public sector governan...
10     [{'code': '10', 'name': 'Rural development'}, ...
11     [{'code': '10', 'name': 'Rural development'}, ...
12     [{'code': '4', 'name': 'Financial and private ...
13     [{'code': '5', 'name': 'Trade and integration'...
14     [{'code': '6', 'name': 'Social protection and ...
15     [{'code': '10', 'name': 'Rural development'}, ...
16     [{'code': '10', 'name': 'Rural development'}, ...
17     [{'code': '8', 'name': '